# Data Loading

In [1]:
X_train_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismic_w_noise_vol_42487393.npy'
y_train_path = '/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismicCubes_RFC_fullstack_2024.42487393.npy'

In [2]:
shape = (1259,300,300)
# load the data
# make the data in 2d section
# make the train set and test set
# print the image also

In [3]:
import os
import pandas as pd
import numpy as np
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
import torch
# 2 files
# 1259*300*300 ---> 1259*300 in 300 images
# x,y load
# x,y sub sample
# pair
# return
class volDataset(Dataset):
    def __init__(self, x_slices, y_slices, transform=None):
        """
        x_slices: List of noisy seismic slices (each of shape 1259x300)
        y_slices: List of clean seismic slices (each of shape 1259x300)
        """
        self.x_slices = x_slices
        self.y_slices = y_slices
        self.transform = transform

    def __len__(self):
        return len(self.x_slices)

    def __getitem__(self, idx):
        """Returns one slice (1, 1259, 300)"""
        x_slice = torch.tensor(self.x_slices[idx], dtype=torch.float32).unsqueeze(0)  # (1, 1259, 300)
        y_slice = torch.tensor(self.y_slices[idx], dtype=torch.float32).unsqueeze(0)  # (1, 1259, 300)

        if self.transform:
            x_slice = self.transform(x_slice)
            y_slice = self.transform(y_slice)

        return x_slice, y_slice

In [4]:
from sklearn.model_selection import train_test_split
# Convert volume to 2D slices
# Load seismic volumes
X_volume = np.load('/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismic_w_noise_vol_42487393.npy',allow_pickle=True)  # (1259, 300, 300)
y_volume = np.load('/home/simlab120/Denoise_comp/Pragyant/image-impeccable-train-data-part1/42487393/seismicCubes_RFC_fullstack_2024.42487393.npy',allow_pickle=True)  # (1259, 300, 300)

# Convert volume to 2D slices

X_slices = [X_volume[:, :, i] for i in range(X_volume.shape[2])]  # List of 300 slices
y_slices = [y_volume[:, :, i] for i in range(y_volume.shape[2])]  # List of 300 slices

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_slices, y_slices, test_size=0.2, random_state=42)

# Create datasets
train_dataset = volDataset(X_train, y_train)
test_dataset = volDataset(X_test, y_test)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Check batch shapes
for x_batch, y_batch in train_loader:
    print("Train Batch Shape:", x_batch.shape, y_batch.shape)  # Expected: (8, 1, 1259, 300)
    break
for x_batch, y_batch in test_loader:
    print("Test Batch Shape:", x_batch.shape, y_batch.shape)  # Expected: (8, 1, 1259, 300)
    break


Train Batch Shape: torch.Size([8, 1, 1259, 300]) torch.Size([8, 1, 1259, 300])
Test Batch Shape: torch.Size([8, 1, 1259, 300]) torch.Size([8, 1, 1259, 300])


# Model

In [5]:
# simple cnn

In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [16]:
class Crop2d(nn.Module):
    def __init__(self, left=10, right=10, top=21, bottom=0):
        super(Crop2d, self).__init__()
        self.left = left
        self.right = right
        self.top = top
        self.bottom = bottom

    def forward(self, x):
        return x[:, :, self.top:x.shape[2] - self.bottom, self.left:x.shape[3] - self.right]


In [17]:
class cnn_model(nn.Module):
    def __init__(self):
        super().__init__()


        self.cnn_model = nn.Sequential(
            nn.ZeroPad2d((10,10,21,0)),
            nn.Conv2d(in_channels=1, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),  # Corrected
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),  # Corrected
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),  # Corrected
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),  # Corrected
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),

            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),  # Corrected
            nn.ReLU(),

            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),  # Corrected
            nn.ReLU(),

            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),  # Corrected
            nn.ReLU(),

            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(in_channels=256, out_channels=1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1),
            nn.ReLU(),
            Crop2d(10, 10, 21, 0),
        )

    def forward(self,x):
        return self.cnn_model(x)
    
model = cnn_model()
x = torch.randn(1,1,1259,300)
output = model(x)

print(output.shape)
        

torch.Size([1, 1, 1259, 300])


# Training

In [18]:
pip install torcheval

Note: you may need to restart the kernel to use updated packages.


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define model
model = cnn_model()

# Define Loss Function (MSE for denoising)
criterion = nn.MSELoss()

# Define Optimizer (Adam works well)
optimizer = optim.Adam(model.parameters(), lr=0.1)



In [20]:
# Check batch shapes
for x_batch, y_batch in train_loader:
    print("Train Batch Shape:", x_batch.shape, y_batch.shape)  # Expected: (batch_size, 1, 1259, 300)
    break


Train Batch Shape: torch.Size([8, 1, 1259, 300]) torch.Size([8, 1, 1259, 300])


In [21]:
from torchmetrics import StructuralSimilarityIndexMeasure
ssim_metric = StructuralSimilarityIndexMeasure()

/home/simlab120/anaconda3/envs/torch/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


In [22]:
num_epochs = 200  # Change as needed

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    running_ssim = 0.0

    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()  # Reset gradients

        outputs = model(x_batch)  # Forward pass
        loss = criterion(outputs, y_batch)  # Compute loss

        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()

        # Compute SSIM (detach to avoid tracking gradients)
        batch_ssim = ssim_metric(outputs.detach(), y_batch.detach())
        running_ssim += batch_ssim.item()




    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, SSIM: {running_ssim/len(train_loader):.4f}")

print("Training complete!")


Epoch [1/200], Loss: 10034.0754, SSIM: 0.4763
Epoch [2/200], Loss: 9583.9085, SSIM: 0.4796
Epoch [3/200], Loss: 8972.9845, SSIM: 0.4891
Epoch [4/200], Loss: 8429.4526, SSIM: 0.4923
Epoch [5/200], Loss: 7921.5783, SSIM: 0.4982
Epoch [6/200], Loss: 7479.4518, SSIM: 0.5031
Epoch [7/200], Loss: 7094.1503, SSIM: 0.5114
Epoch [8/200], Loss: 6732.0171, SSIM: 0.5203
Epoch [9/200], Loss: 6416.2509, SSIM: 0.5284
Epoch [10/200], Loss: 6132.7325, SSIM: 0.5367
Epoch [11/200], Loss: 5894.2965, SSIM: 0.5451
Epoch [12/200], Loss: 5674.0245, SSIM: 0.5500
Epoch [13/200], Loss: 5486.1499, SSIM: 0.5579
Epoch [14/200], Loss: 5305.1999, SSIM: 0.5666
Epoch [15/200], Loss: 5150.6497, SSIM: 0.5718
Epoch [16/200], Loss: 4997.5051, SSIM: 0.5788
Epoch [17/200], Loss: 4892.1826, SSIM: 0.5820
Epoch [18/200], Loss: 4781.8391, SSIM: 0.5879
Epoch [19/200], Loss: 4677.4684, SSIM: 0.5941
Epoch [20/200], Loss: 4613.4869, SSIM: 0.5951
Epoch [21/200], Loss: 4512.7434, SSIM: 0.6016
Epoch [22/200], Loss: 4449.8108, SSIM: 0.6